In [1]:
from classes import *
from Baseline import *
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

Resizing Masks: 100%|██████████| 48/48 [00:00<00:00, 7443.03it/s]


 (768, 1024, 3) -> (192, 256, 3)


In [2]:
pieces = np.array(Puzzle(MATCH_IMGS).pieces)

Generating Pieces: 48it [00:10,  4.59it/s]


In [3]:
#Get test and train features
def get_DoG_train(pieces):
    foreground = []
    background = []
    for p in pieces:
        foreground.append(np.concatenate([p.RGB_foreground, p.DoG_foreground], axis = 1))
        background.append(np.concatenate([p.RGB_background, p.DoG_background], axis = 1 ))
    foreground = np.concatenate(foreground)
    background = np.concatenate(background)
    
    return foreground, background
  
def get_DoG_test(pieces):
    X_test = []
    msks = []
    for p in pieces:
        X_test.append(np.concatenate([p.features_RGB, p.features_DoG], axis = 1))
        msks.append(p.mask.flatten())
    return np.concatenate(X_test), np.concatenate(msks)

def get_RGB_train(pieces):
    foreground = []
    background = []
    for p in pieces:
        foreground.append(p.RGB_foreground)
        background.append(p.RGB_background)
    foreground = np.concatenate(foreground)
    background = np.concatenate(background)
    
    return foreground, background

def get_RGB_test(pieces):
    X_test = []
    msks = []
    for p in pieces:
        X_test.append(p.features_RGB)
        msks.append(p.mask.flatten())
    return np.concatenate(X_test), np.concatenate(msks)


def get_MR8_train(pieces):
    foreground = []
    background = []
    for p in pieces:
        foreground.append(p.MR8_foreground)
        background.append(p.MR8_background)
    foreground = np.concatenate(foreground)
    background = np.concatenate(background)
    
    return foreground, background

    
def get_MR8_test(pieces):
    X_test = []
    msks = []
    for p in pieces:
        X_test.append(p.features_MR8)
        msks.append(p.mask.flatten())
    return np.concatenate(X_test), np.concatenate(msks)

#Check if predictions make sense
def plot_prob(height = 192, width = 256, p = None, k= 0 ):
    plt.imshow(p[k*height*width:(k+1)*height*width].reshape(height, width))

In [4]:
train = pieces[:40]
test = pieces[40:]

In [5]:
#Train foreground and background model
foreground_RGB, background_RGB = get_MR8_train(train)
X_RGB, y = get_MR8_test(test)
Model_RGB = Baseline()
Model_RGB.fit(foreground_RGB)
RGBF_pred = Model_RGB.predict(X_RGB)
Model_RGB.fit(background_RGB)
RGBB_pred = Model_RGB.predict(X_RGB)
l = len(foreground_RGB) / ( len(foreground_RGB) + len(background_RGB) )

In [6]:
def run(features_Train = None, features_Test = None,  threshold = 0.5, k = 6, pieces = None):
    """
    features_Train : Function to extract train features
    features_Test: Function to extract test features
    """
    kfold = KFold(6)
    scores = []
    for train, test in kfold.split(pieces):
        foreground, background = features_Train(pieces[train])
        X, y = features_Test(pieces[test])
        Base_model = Baseline()
        Base_model.fit(foreground)
        pred_f = Base_model.predict(X)
        Base_model.fit(background)
        pred_b = Base_model.predict(X)
        l = len(foreground) / ( len(background) + len(foreground) )
        pred = pred_f*l / (l*pred_f + (1-l)*pred_b)
        scores.append(roc_auc_score(pred > threshold, y))
        
    return np.mean(scores)

In [12]:
print("RGB features")
score = run(features_Train = get_RGB_train, features_Test = get_RGB_test,  threshold = 0.5, k = 6, pieces = pieces)
print("Mean RGB score: ", score)
print("RGB & DoG features")
score = run(features_Train = get_DoG_train, features_Test = get_DoG_test,  threshold = 0.5, k = 6, pieces = pieces)
print("Mean DoG score:", score)
print("MR8 features")
score = run(features_Train = get_MR8_train, features_Test = get_MR8_test,  threshold = 0.5, k = 6, pieces = pieces)
print("Mean MR8 Score:", score)

RGB features
Mean RGB score:  0.9447201176152941
RGB & DoG features
Mean DoG score: 0.9307032476121422
MR8 features
Mean MR8 Score: 0.8969537419208967


/Library/Python/3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]
/Library/Python/3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in greater
  
